In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show

import sklearn.preprocessing as preprocessing

%matplotlib inline
sns.set_context('notebook')
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500) # to see all columns

In [2]:

data_agr = pd.read_csv('CrowdstormingDataJuly1st_aggregated_encoded.csv')
data_agr    

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,defeats,goals,yellowCards,yellowReds,redCards,rater1,rater2,refCount,refCountryCount,meanIAT,seIAT,meanExp,seExp,color_rating,meanIAT_nIAT,meanExp_nExp,meanIAT_GameNbr,meanExp_GameNbr
0,0,392,34,0,360,182.0,71.0,1,654,247,179,228,9,19,0,0,0.25,0.00,166,37,0.346459,0.003297,0.494575,0.013752,1,0.328409,0.367721,0.333195,0.400637
1,1,393,91,2,176,183.0,73.0,0,336,141,73,122,62,42,0,1,0.00,0.25,99,25,0.348818,0.003297,0.449220,0.013752,2,0.329945,0.441615,0.341438,0.380811
2,2,394,83,0,719,165.0,63.0,11,412,200,97,115,31,11,0,0,0.25,0.25,101,28,0.345893,0.003297,0.491482,0.013752,2,0.328230,0.365628,0.332389,0.399459
3,3,395,6,0,1199,178.0,76.0,3,260,150,42,68,39,31,0,1,0.00,0.00,104,37,0.346821,0.003297,0.514693,0.013752,1,0.327775,0.412859,0.336638,0.433294
4,4,396,51,1,758,180.0,73.0,1,124,41,40,43,1,8,4,2,0.25,0.25,37,11,0.331600,0.000151,0.335587,0.000586,2,0.338847,0.379497,0.331882,0.328895
5,5,397,36,1,751,180.0,74.0,11,97,41,23,33,3,11,1,0,0.75,0.75,42,13,0.320079,0.000229,0.296562,0.001002,4,0.332117,0.366031,0.327985,0.317247
6,6,398,80,1,1341,189.0,80.0,5,24,8,8,8,0,0,0,0,0.75,1.00,20,5,0.341625,0.000151,0.400818,0.000586,4,0.337572,0.361394,0.343556,0.428271
7,7,399,36,1,872,188.0,83.0,5,3,0,1,2,0,0,0,0,0.75,1.00,2,2,0.355406,0.000151,0.417225,0.000586,5,0.341567,0.363284,0.348498,0.390184
8,8,400,78,1,897,180.0,68.0,4,121,34,25,62,3,21,0,2,1.00,1.00,56,3,0.348178,0.000151,0.429630,0.000586,5,0.344426,0.404517,0.344130,0.402314
9,9,401,77,2,82,179.0,70.0,7,50,17,8,25,0,3,0,1,0.75,1.00,27,7,0.342072,0.000055,0.361068,0.000225,4,0.337478,0.339232,0.339733,0.341395


## KMeans

In [3]:
from sklearn import metrics
from sklearn.cluster import KMeans

In [4]:
np.random.seed(1)
nbr_class = 2

In [5]:
kmeans = KMeans(init='k-means++', n_clusters=nbr_class, n_init=1)
kmeans.fit(data_agr)
metrics.silhouette_score(data_agr, kmeans.labels_, metric='euclidean')

0.3460965800514027

# 3. Machine learning

In [6]:
# encoders
le = pp.LabelEncoder() # for playernames etc.
ohe = pp.OneHotEncoder() # for categories with 'few' categories

NameError: name 'pp' is not defined

In [ ]:
rf_input_data = data_agr.copy()

rf_input_data['player'] = le.fit_transform(data_agr['player'])
rf_input_data['club'] = le.fit_transform(data_agr['club'])
rf_input_data['leagueCountry'] = le.fit_transform(data_agr['leagueCountry'])
rf_input_data['birthday'] = le.fit_transform(data_agr['birthday'])
rf_input_data['position'] = le.fit_transform(data_agr['position'])
rf_input_data['color_rating'] = le.fit_transform(data_agr['color_rating'])
rf_input_data.head(1)

In [ ]:
player_colors = rf_input_data['color_rating']
rf_input_data = rf_input_data.drop(['rater1', 'rater2', 'color_rating', 'playerShort'], axis=1) 


rf_input_data.dtypes

In [ ]:
rf = RandomForestClassifier()
# roc_scorer = make_scorer(roc_curve)
X = rf_input_data
print(rf_input_data.columns)
y = player_colors
result = cross_val_score(rf, X, y, cv=10, n_jobs=3, pre_dispatch='n_jobs+1', verbose=3)

"""
kf = KFold(n_splits=10)
for train, test in kf.split(rf_input_data):
    X_train, X_test, y_train, y_test = rf_input_data[train], rf_input_data[test], player_colors[train], player_colors[test]
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
"""

result

In [ ]:
rf.fit(X, y)
    
    
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

cols = rf_input_data.columns
# Print the feature ranking
print("Feature ranking:")
for f in range(X.shape[1]):
    print("%d. feature n° %d %s (%f)" % (f + 1, indices[f], cols[indices[f]], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()    